<a href="https://colab.research.google.com/github/ronaldehido/EchoSonR/blob/main/BirdNET_running/BirdNET-CLI_custom-WildmonV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://raw.githubusercontent.com/birdnet-team/BirdNET-Analyzer/refs/heads/main/docs/_static/birdnet_logo.png" width="50">**Inferencias con BirdNET-CLI sobre audios en Google Drive**<img src="https://upload.wikimedia.org/wikipedia/commons/1/12/Google_Drive_icon_%282020%29.svg" width="50"> 
## **Inferencia con modelos personalizados  y caso Wildmon V3 y FLAT SIGMOIDS)**
_versión 1 (2025-07-25)_



### *Ron A. Fernández-Gómez - EPM SECIHTI* 🤓
<img src="https://upload.wikimedia.org/wikipedia/commons/1/17/Logotipo_SECIHTI_2025-2030.svg" width="200">

Este es un flujo de trabajo en Google Colab escrito en Python para ejecutar clasificadores personalizados basados en Redes Neuronales Convolucionales (CNNs) para detección y clasificación de vocalizaciones de fauna utilizando la herramienta BirdNET Analyzer para uso con Interfaz de línea de comando (CLI) y accediendo a archivos de audio alojados en Google Drive, Unidades Compartidas para trabajo colaborativo, asi como a partir de audios de otras fuentes locales.
Para mayores detalles sobre BirdNET Analyzer y usos del proyecto BirdNET consultar: https://github.com/birdnet-team/BirdNET-Analyzer


##### **_Comentarios sobre clasificador personalizado Wildmon V3_**
La versión V3 del clasificador genera outputs que distribuyen los scores en un rango entre 0.5 y 0.7, para ampliar la distribución a toda la escala de los confidence scores, se evita la aplicación de la sigmoides sobre las predicciones. Para esto fue necesario modificar el codigo fuente por lo que el analizador utilizado es un repositorio clonado modificado.
El cambio especifico para saltar las sigmoides se hizo en el archivo config.py aplicando el operador booleano False en el argumento APPLY_SIGMOID (https://github.com/birdnet-team/BirdNET-Analyzer/blob/main/birdnet_analyzer/config.py#L85)



# 1. INFERENCIA: Deteccion de vocalizaciones por lotes
Esta etapa corresponde a la ejecución de un modelo CNN por lotes. Permite ejecutar el modelo en una carpeta que contenga varios archivos de audio.

## Paqueterías y dependencias necesarias
Estas dependencias facilitaran el funcionamiento del analizador.

In [ ]:
!pip install ffmpeg
!pip install librosa
!pip install resampy
!pip install tensorflow==2.15.0
!pip install keras-tuner

## *Conectar a Google Drive y definir rutas*
Primero vincularemos Google Drive con lo que definirimos las rutas de acceso a los archivos de trabajo y de salida de resultados durante la sesión de trabajo. En el Drive deberán estar los archivos de audio a analizar.

In [ ]:
# Conectar con google drive y montar la unidad
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Rutas de entrada y salida a tu Google Drive
audio_path = '/ruta/a/la/carpeta/de/archivos' # definir la ruta al directorio con los archivos de audio
output_folder = '/ruta/a/la/carpeta/donde/guardar/los/resultados' # definir la ruta al directorio donde se guardarán los resultados

In [ ]:
# establecer el nombre del sitio y/o muestreo
site_name = 'nombre_del_sitio'  # <--- Cambia esto al nombre del sitio que estás analizando, por ejemplo, 'R405-202408'

### Modelo personalizado
En este caso usaremos el modelo V3 de WildMon. Comparto el modelo el cual esta etiquetado de forma que pueda ser reconocido por el analyzador de BirdNET y tambien se reconozcan las especies. Será necesario clonar el repositorio de EchoSonR para acceder al Model-Zoo donde adicionalmente encontrará otras versiones de este y otros modelos pre-entrenados.

In [ ]:
### clonamos el Model-Zoo de EchoSonR para acceder al clasificador personalizado
!mkdir -p /content/Model-Zoo
%cd /content/Model-Zoo
!git init
!git remote add origin https://github.com/ronaldehido/EchoSonR.git
!git config core.sparseCheckout true
!echo "BirdNET_running/Model-Zoo/*" > .git/info/sparse-checkout
!git pull origin main
!mv BirdNET_running/Model-Zoo/* ./
!rm -rf BirdNET_running
%cd /content

In [ ]:
# Ruta al modelo (en caso de usar un modelo personalizado)
# modelo V3 de Wildmon
my_model = '/content/Model-Zoo/WildMon/Wildmon-V3/WildMon-20spp-V3.0_Model_FP32.tflite' #definir la ruta

### *Clonar BirdNET Analyzer*
Clonamos el respositorio de BirdNET Analyzer en nuestro espacio de trabajo desde GitHub. Esto nos permite ejecutar el analizador y sus funciones. Aquí presento dos opciones de repositorios clonados. 1) la primera opciones es clonar el repositorio directamente desde el GitHub del BirdNET-Team. La opcion 2) es un repositorio clonado de BirdNET a la fecha de 2024.07.25 pero modificado por Ron A. Fernández-Gómez para que no se apliquen las sigmoides sobre las predicciones. Esto es útil para que el modelo V3 de Wildmon pueda generar predicciones con scores distribuidos en toda la escala (0 a 1). Escoge la opción que corresponda en tu caso.

**OPCIÓN 1)** Clonar BirdNET-Analyzer desde el respositorio original del BirdNET Team

In [ ]:
# OPCIÓN 1) Clonar BirdNET-Analyzer desde el respositorio original del BirdNET Team
!git clone https://github.com/birdnet-team/BirdNET-Analyzer.git

**OPCIÓN 2)** Clonar la version modificada de BirdNET-Analyzer

In [ ]:
# OPCIÓN 2) Clonar la version modificada de BirdNET-Analyzer
!git clone https://github.com/ronaldehido/EchoSonR/tree/main/BirdNET_running/BirdNET-repos/BirdNET-Analyzer_Mod_runWMV3

In [ ]:
# se establece el nuevo directorio de trabajo donde estan las funciones
# para la version modificada de BirdNET-Analyzer será:
%cd BirdNET-Analyzer_Mod_runWMV3

# Para la version original de BirdNET-Analyzer usar:
# %cd BirdNET-Analyzer

### *Ejemplo de inferencia con un solo archivo de audio*
Usamos un ejemplo incluido en BirdNET para hacer una prueba rápida de que la función corre correctamente. La prueba se usa corriendo el modelo base (GLOBAL V2.4). El resultado se obtendrá como un archivo de texto en la carpeta de trabajo. Lo revisamos y verificamos que todo sea correcto.

In [ ]:
#Hacemos una prueba rapida con un ejemplo incluido (archivo example) para verificar el funcionamiento correcto de la funcion "analyze"
!python -m birdnet_analyzer.analyze birdnet_analyzer/example/ --slist birdnet_analyzer/example/species_list.txt --min_conf 0.5 --threads 4 --combine_results

### *Inferencia en lote con modelo personalizado*
Ejecutamos BirdNET sobre un directorio completo y ajustamos parametros adicionales segun nuestra necesidad, además incluimos un modelo entrenado personalizado. En caso de querer usar el modelo por defecto (GLOBAL) no se declara el argumento --classifier

In [ ]:
# Procesar una carpeta con archivos y el modelo personalizado, en este caso cargamos el modelo de WildMon dando la ruta del archivo
# --classifier es la ruta al modelo personalizado, si no se especifica se usa el modelo por defecto (GLOBAL)
# --min_conf aquí lo ajuste al mínimo para obtener el total de detecciones, si se quiere un umbral mayor se puede ajustar
!python -m birdnet_analyzer.analyze \
        --output "{output_folder}" \
        --classifier "{my_model}" \
        --min_conf 0.0001 \
        --threads 2 \
        --b 1 \
        --combine_results \
        "{audio_path}"

# Otros argumentos que se pueden usar:
# usage: birdnet_analyzer.analyze [-h] [-o OUTPUT] [--fmin FMIN] [--fmax FMAX]
#                                [--lat LAT] [--lon LON] [--week WEEK]
#                                [--sf_thresh SF_THRESH] [--slist SLIST]
#                                [--sensitivity SENSITIVITY]
#                                [--overlap OVERLAP]
#                                [--audio_speed AUDIO_SPEED] [-t THREADS]
#                                [--min_conf MIN_CONF] [-l LOCALE]
#                                [-b BATCH_SIZE]
#                                [--rtype {table,audacity,kaleidoscope,csv} [{table,audacity,kaleidoscope,csv} ...]]
#                                [--additional_columns {lat,lon,week,overlap,sensitivity,min_conf,species_list,model} [{lat,lon,week,overlap,sensitivity,min_conf,species_list,model} ...]]
#                                [--combine_results] [-c CLASSIFIER]
#                                [--skip_existing_results] [--top_n TOP_N]
#                                [--merge_consecutive MERGE_CONSECUTIVE]
#                                INPUT


# Mostrar resumen al final
import os

print("\n ======== RESUMEN DE LA PREDICIÓN: ========")
print(f"📁 Modelo usado: {os.path.basename(my_model)}")
print(f"📂 Carpeta de resultados: {output_folder}")
print(f"📍 Sitio analizado: {site_name}")
print(f"🎯 Valor de confianza mínimo: {min_conf}")

### **Fin.**